id: unique id for a news article 
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks the article as potentially unreliable
1: unreliable
0: reliable

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [3]:
import os 
os.chdir('C:\\Data\\deepmodel')

In [4]:
data_train=pd.read_csv('train.csv')
data_test= pd.read_csv('test.csv')

In [5]:
data_train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
data_train['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [7]:
data_train=data_train.dropna()

In [8]:
X=data_train.drop('label',axis=1)
y=data_train['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [12]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
 
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Embedding Representation


In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1387 2534  588]
 [   0    0    0 ... 3768 1068  726]
 [   0    0    0 ... 1769 3734 1237]
 ...
 [   0    0    0 ...  335 2428 3294]
 [   0    0    0 ... 1036 3846  913]
 [   0    0    0 ...  967  662 2059]]


In [23]:

embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4667,
       4731,  913,  764,  274, 2671, 4183, 1387, 2534,  588])

In [24]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 30ms/step - loss: 0.3405 - accuracy: 0.8394 - val_loss: 0.2060 - val_accuracy: 0.9125
Epoch 2/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1515 - accuracy: 0.9407 - val_loss: 0.1919 - val_accuracy: 0.9206
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1075 - accuracy: 0.9604 - val_loss: 0.2143 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0741 - accuracy: 0.9738 - val_loss: 0.2331 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0534 - accuracy: 0.9815 - val_loss: 0.2914 - val_accuracy: 0.9162
Epoch 6/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0408 - accuracy: 0.9860 - val_loss: 0.3251 - val_accuracy: 0.9173
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0272 - accuracy: 0.9923 - val_loss: 0.3442 - val_accuracy: 0.9060

In [30]:
y_pred=model.predict_classes(X_test)

C:\Anaconda folder\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9105219552609777